In [13]:
%%writefile aula11.cu
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/reduce.h>
#include <thrust/replace.h>
#include <thrust/functional.h>
#include <iostream>
#include <fstream>
#include <vector>
#include <sstream>

// Functor to compute the difference
struct difference {
    __host__ __device__
    float operator()(const float &x, const float &y) const {
        return x - y;
    }
};

// Functor to check for positive increases
struct is_positive {
    __host__ __device__
    bool operator()(const float &x) const {
        return x > 0;
    }
};

// Functor to calculate squared differences for variance
struct square_difference {
    const float mean;
    square_difference(float mean) : mean(mean) {}

    __host__ __device__
    float operator()(const float &x) const {
        return (x - mean) * (x - mean);
    }
};

// Functor to check for non-positive values (used in thrust::replace_if)
struct is_non_positive {
    __host__ __device__
    bool operator()(const float &x) const {
        return x <= 0;
    }
};

// Function to read stock prices from a file
std::vector<float> read_prices_from_file(const std::string& filename) {
    std::vector<float> prices;
    std::ifstream file(filename);
    std::string line;

    while (std::getline(file, line)) {
        std::stringstream ss(line);
        float price;
        if (ss >> price) {
            prices.push_back(price);
        }
    }

    return prices;
}

int main() {
    // Read prices from file
    std::vector<float> host_prices = read_prices_from_file("stocks-google.txt");

    // Copy data to device vector
    thrust::device_vector<float> prices(host_prices.begin(), host_prices.end());
    thrust::device_vector<float> diffs(prices.size() - 1);

    // Compute differences between consecutive prices
    thrust::transform(prices.begin() + 1, prices.end(), prices.begin(), diffs.begin(), difference());

    // Calculate mean of differences
    float sum_diffs = thrust::reduce(diffs.begin(), diffs.end(), 0.0f, thrust::plus<float>());
    float mean_diffs = sum_diffs / diffs.size();

    // Calculate variance of differences
    thrust::device_vector<float> squared_diffs(diffs.size());
    thrust::transform(diffs.begin(), diffs.end(), squared_diffs.begin(), square_difference(mean_diffs));
    float variance = thrust::reduce(squared_diffs.begin(), squared_diffs.end(), 0.0f, thrust::plus<float>()) / diffs.size();

    // Count positive differences (price increases)
    int count_increases = thrust::count_if(diffs.begin(), diffs.end(), is_positive());

    // Replace non-positive differences with zero using the custom functor
    thrust::replace_if(diffs.begin(), diffs.end(), is_non_positive(), 0.0f);

    // Calculate sum of positive differences
    float total_increase = thrust::reduce(diffs.begin(), diffs.end(), 0.0f, thrust::plus<float>());
    float average_increase = (count_increases > 0) ? total_increase / count_increases : 0.0f;

    // Output results
    std::cout << "Já vimos que, para valer a pena usar a GPU, precisamos tratar grandes volumes de dados ou realizar cálculos complexos." << std::endl;
    std::cout << "Número de aumentos no preço das ações: " << count_increases << std::endl;
    std::cout << "Aumento médio nos dias em que o preço subiu: " << average_increase << std::endl;
    std::cout << "Variância das diferenças diárias: " << variance << std::endl;

    return 0;
}


Overwriting aula11.cu


In [14]:
!nvcc -arch=sm_70 -std=c++14 aula11.cu -o stocks

In [15]:
!./stocks < stocks-google.txt

Já vimos que, para valer a pena usar a GPU, precisamos tratar grandes volumes de dados ou realizar cálculos complexos.
Número de aumentos no preço das ações: 3056
Aumento médio nos dias em que o preço subiu: 3.15251
Variância das diferenças diárias: 3168.44
